<a href="https://colab.research.google.com/github/MiuraKunihiro/Cats_vs_Dogs/blob/master/keras_%E6%95%99%E6%9D%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Kerasを使った猫と犬の分類**

# はじめに
今回はbackendにTensorflowを使いつつ、Kerasによる機械学習を行います。使用したデータは猫と犬の写真です。

(data source:https://www.kaggle.com/c/dogs-vs-cats/data)

# 1.データの読み込み

データソースににあるオリジナルのデータには猫と犬それぞれ12500枚以上の写真がありましたが、

今回はその中のごく一部の写真を使って機械学習を行なっていきます。

google drive にその必要な写真データを用意したので、これを使います。


まずは必要なパッケージを取得します。

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

以下のコードを実行すると、青いURLが出現します。

クリックした移動先でアカウントにログインし、許可ボタンを押して下さい。

その次に出現するリンクをコピーして、以下のボックスにペーストし、エンターして下さい。

In [0]:
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

以下のコードを実行し、先ほどと同じようにログイン→許可→リンクをコピー→ボックスにペーストと一連の操作をおこなって下さい。

In [0]:
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

以下のコマンドを実行することによってgoogle driveの特定フォルダにマウントして目的のデータにアクセスできるようになります。

In [0]:
!cp -f ~/.gdfuse/default/config config
!sed -i -e "s/^root_folder=$/root_folder=1amEIqdAv4_C44JoaerIwqfE-D-nTltxi/" config

!mkdir -p drive
!google-drive-ocamlfuse -config ./config -cc drive
!ls drive

# 2.読み込んだ各フォルダにパスを通す

読み込んだフォルダの構造と内容は以下のようになっています。



```
Cars_vs_Dogs
  │ 
  ├── train
  │   ├── cats 100枚の写真（cat.0.jpg ~ cat.99.jpg）
  │   └── dogs 100枚の写真（dog.0.jpg ~ dog.99.jpg）
  ├── validation
  │   ├── cats 50枚の写真（cat.100.jpg ~ cat.149.jpg）
  │   └── dogs 50枚の写真（dog.100.jpg ~ dog.149.jpg）
  └── test
      ├── cats 50枚の写真（cat.150.jpg ~ cat.199.jpg）
      └── dogs 50枚の写真（dog.150.jpg ~ dog.199.jpg）
```



以下のコードを実行して下さい。

パスに対応した変数が用意できます。

In [0]:
train_dir = '/content/drive/train'
train_cats_dir = '/content/drive/train/cats'
train_dogs_dir = '/content/drive/train/dogs'
validation_dir = '/content/drive/validation'
validation_cats_dir = '/content/drive/validation/cats'
validation_dogs_dir = '/content/drive/validation/dogs'
test_dir = '/content/drive/test'
test_cats_dir = '/content/drive/test/cats'
test_dogs_dir = '/content/drive/test/dogs'

# 3.データの前処理

学習を行えるようにデータを編集します。

まずimage.ImageDataGenerator オブジェクトを作成する。この際にオプションを指定してRGB(0〜255)で読み込まれた各画素のRGB値を0.0〜1.0の間に収まるように正規化したり、ランダムに拡張してデータの水増しができます。

次にflow_from_directory()　メソッドで指定したフォルダーの画像をリサイズや正解ラベル付けを行います。

## 例

In [0]:
from keras.preprocessing.image import ImageDataGenerator

#画像データの拡張パラメータを設定
datagen = ImageDataGenerator(
    rotation_range=0., 　　　# 画像をランダムに回転する回転範囲（0-180）
    width_shift_range=0., 　　# ランダムに水平シフトする範囲
    height_shift_range=0., 　　# ランダムに垂直シフトする範囲
    shear_range=0.2,　　　　 # シアー強度（反時計回りのシアー角度（ラジアン））
    zoom_range=0.2, 　　　　　# ランダムにズームする範囲
    horizontal_flip=True, 　　# 水平方向に入力をランダムに反転
    vertical_flip=True, 　　# 垂直方向に入力をランダムに反転
    rescale=1.0 / 255,　　 # 各画素のRGB値を正規化
    )

#flow_from_directoryで指定したディレクトリから一枚ずつ画像を取り出す
train_generator = train_datagen.flow_from_directory(
        train_dir,　　　　　　　　　　#指定するディレクトリ
        target_size=(250, 250),　　#縦、横のサイズを指定してリサイズ
        batch_size=20,　　　　　　 　#ミニバッチ学習に使われるバッチのサイズ指定
        class_mode="categorical"　　#正解ラベルの生成の仕方
)


## 実習1

"###" **を適切なコードで置き換えて下さい。**

In [0]:
from keras.preprocessing.image import ImageDataGenerator
 
#訓練画像は水増しする
#好きなパラメータを入れて下さい　デフォルトは0かFALSEです
train_datagen = ImageDataGenerator(rescale=###
                                   ###
                                   ###
                                   ###
                                   )

validation_datagen = ImageDataGenerator(rescale=1./255)
 
train_generator = ###(
    train_dir,
    ###,
    ###,
    ###"
)
 
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    batch_size=10,
    class_mode="binary"
)

##解答例

In [0]:
from keras.preprocessing.image import ImageDataGenerator
 
train_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode="nearest"
                                    )

validation_datagen = ImageDataGenerator(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=10,
    class_mode="binary"
)
 
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    batch_size=10,
    class_mode="binary"
)

データとラベルの型を確認します。モデルはこの型を参考にして作成します

In [0]:
for data,label in train_generator:
    print(data.shape)
    print(label.shape)
    break
print(train_generator.class_indices)

# 4.CNNモデルの作成


<img src = "https://kenyu-life.com/wp-content/uploads/2019/03/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88-2019-03-07-8.56.07.jpg" alt = "filter" height = "250px">

<small>画像は[kenyu-life.com](https://kenyu-life.com/2019/03/07/convolutional_neural_network/) から引用 </small>

CNN（畳み込みニューラルネットワーク）では、


1. 畳み込みフィルタ層：画像の濃淡パターンを検出します。（エッジ抽出等の特徴抽出）

<img src = "https://kenyu-life.com/wp-content/uploads/2019/03/3cnn.gif" alt = "filter" height = "400px">

<small>画像は[kenyu-life.com](https://kenyu-life.com/2019/03/07/convolutional_neural_network/) から引用 </small>

2. プーリング層：物体の位置が変動しても同一の物体であると見なせるようにします。（位置ズレを許容する）また，プーリングには，「maxプーリング」と「avgプーリング」の2種類があります．

<img src = "https://kenyu-life.com/wp-content/uploads/2019/03/pooling.gif" alt = "filter" height = "200px">

<small>画像は[kenyu-life.com](https://kenyu-life.com/2019/03/07/convolutional_neural_network/) から引用 </small>

　これらの層を組み合わせることによって、画像から特徴量を抽出する働きを担っています。


　一方、特徴量を抽出するだけでは、画像の識別はできません。識別には、「特徴量に基づいた分類」が必要です。この分類の役割を担っているのが、全結合層と出力層です。


3. 全結合層：①②を通して特徴部分が取り出された画像データを一つのノードに結合し、活性化関数（後述）によって変換された値（特徴変数）を出力します。ノードの数が増えると特徴量空間の分割数が増し、各領域を特徴付ける特徴変数の数が増えます。

4. 出力層：全結合層からの出力（特徴変数）を元に、ソフトマックス関数を用いて確率に変換し、それぞれの領域に正しく分類される確率を最大化する（最尤推定法）ことによって分類を行います。



## 例

In [0]:
from keras import layers
from keras import models
 
model = models.Sequential()
model.add(layers.Conv2D(32,       #フィルターの数
                        (3,3),      #3x3のフィルター
                        activation="relu",     #活性化関数
                        input_shape=(150,150,3))   #入力データの型をここで一度指定する
)
model.add(Conv2D(32,(3,3),activation="relu")
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),activation="relu")
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(1024,activation="relu")
model.add(Activation('relu'))
model.add(Dropout(1.0))

model.add(Dense(nb_classes, activation='softmax'))

## 実習2

以下の画像を参考に、畳込みが3層の小さなモデルを構成したい。(レイヤーを増やすと、１つ目に適用した結果に対して２回目の畳み込みをするため、より小さい特徴を捉えられるようになる)


青で囲った部分が畳込み層（Convolution2D）+プーリング層（MaxPooling2D）で緑で囲った部分がフル結合層です。

入力画像を150x150ピクセルで3チャンネルとした時のアウトプットを表している。



<img src = "https://cdn-ak.f.st-hatena.com/images/fotolife/a/aidiary/20170110/20170110195058.png" alt = "CNN" height = "400px">

<small>画像は[Hatena Blog](http://aidiary.hatenablog.com/entry/20170110/1484057655) から引用 </small>

"###" を適切なコードで置き換えて下さい。

In [0]:
from keras import layers
from keras import models
 
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation="relu",input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
 
model.add(###(64,(3,3),###)
model.add(###((2,2)))
 
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
 
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
 
model.add(layers.Flatten())
 
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512,activation="relu"))
model.add(layers.Dense(1,activation="sigmoid"))
 
model.summary()

## 解答例

In [0]:
from keras import layers
from keras import models
 
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation="relu",input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
 
model.add(layers.Conv2D(64,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
 
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
 
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
 
model.add(layers.Flatten())
 
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512,activation="relu"))
model.add(layers.Dense(1,activation="sigmoid")) #2値分類なので最終出力は1つのニューロン
 
model.summary()

# 5.コンパイル

モデルを使った学習の前に、どのような学習処理を行うかをcompileメソッドを用いて設定します。

## 例

In [0]:
from keras import optimizers

model.compile(loss="mean_squared_error",　　##ズレの大きさを定量化する損失関数
             optimizer=optimizers.RMSprop(lr=1e-4),   #最適化アルゴリズム
             metrics=["acc"])   #訓練やテストの際にモデルを評価するための評価関数のリスト

## 実習3
"###" を適切なコードで置き換えて下さい。
ただし損失関数は"binary_crossentropy"を指定します。

In [0]:
from keras import optimizers

model.compile(###,　
             optimizer="adam",   
             metrics=["acc"]) 

## 解答例

In [0]:
from keras import optimizers

model.compile(loss="binary_crossentropy",
             optimizer="adam",   
             metrics=["acc"]) 

# 6.モデルの学習



Kerasで作ったモデルをfitすると、戻り値として損失関数や正解率を格納したHistoryオブジェクトが返されます。

In [0]:
history = model.fit_generator(train_generator,
                             steps_per_epoch=30,
                             epochs=5,
                             validation_data=validation_generator,
                             validation_steps=10)

# 7.学習した過程を可視化

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history["acc"]
val_acc = history.history["val_acc"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1,len(acc) + 1)

plt.plot(epochs, acc,"bo",label="Training Acc")
plt.plot(epochs, val_acc,"b",label="Validation Acc")
plt.legend()

plt.figure()

plt.plot(epochs,loss,"bo",label="Training Loss")
plt.plot(epochs,val_loss,"b",label="Validation Loss")
plt.legend()

plt.show()

# 8.テストデータを使って評価

trainingやvalidationとは別に用意したtest用画像を使って評価してみます。

In [0]:
# テスト用画像データのジェネレータ
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150,150),
    batch_size=1,
    class_mode=None,
    shuffle=False
)

# 分類予測
pred = model.predict_generator(
        test_generator,
        steps=100,
        verbose=1
)

In [0]:
import numpy as np

labels = ['cat', 'dog']

print("*** test data [cat] *****")
for i in pred[0:15]:
    cls = np.argmax(i)
    score = np.max(i)
    print("pred: {}  score = {:.3f}".format(labels[cls], score))

print("-" * 30)

print("*** test data [dog] *****")
for i in pred[15:30]:
    cls = np.argmax(i)
    score = np.max(i)
    print("pred: {}  score = {:.3f}".format(labels[cls], score))